In [15]:
import cv2
import numpy as np


# Preprocessing of the image
# Don't ask, it just works
image = cv2.imread('test3.jpg')

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (3, 3), 0)
canny = cv2.Canny(blurred, 120, 255, 1)


# Find contours
contours, hierarchy = cv2.findContours(canny, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Iterate thorugh contours and draw rectangles around all the contours found
#for c in contours:
#    x,y,w,h = cv2.boundingRect(c)
#    cv2.rectangle(image, (x, y), (x + w, y + h), (36,255,12), 2)

maxArea = -1
for c in contours:
    if cv2.contourArea(c) > maxArea:
        cMax = c
        maxArea = cv2.contourArea(c)


# Find the (rotated) rectangle with the minimum area
rect = cv2.minAreaRect(c)
area = cv2.contourArea(c) # area is needed later to find the rotate image once again
#print(area)
box = np.int0( cv2.boxPoints(rect) ) # Get vertices coordinates and transform them in integers


# Draw the box and show what we have so far
#cv2.drawContours(image,[box],0,(0,0,255),2)
#print(box)
#cv2.imshow('canny', canny)
#cv2.imshow('image', image)
#cv2.imwrite('canny.png', canny)
#cv2.waitKey(0)


# Rotate the image
# Box vertices are given counter-clockwise
dy = box[0][1] - box[1][1]
dx = box[0][0] - box[1][0]
angle = np.arctan(dy/dx)*180/np.pi

# Center of the rectangle in order to perform the rotation
cX = np.abs( (box[0][0] + box[2][0])//2 )
cY = np.abs( (box[0][1] + box[2][1])//2 )

rot_mat = cv2.getRotationMatrix2D( (cX, cY), angle, 1.0)
result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)

# Show intermediate results
#cv2.circle(img, (cX, cY), radius=20, color=(255, 170, 29), thickness=-1)
#cv2.imshow('image', image)
#cv2.imshow('result', result)
#cv2.waitKey(0)

In [16]:
# -----------------------------------------------
# REPEAT THE PROCESS AND CROP THE CARD
# -----------------------------------------------


gray = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (3, 3), 0)
canny = cv2.Canny(blurred, 120, 255, 1)


# Find contours
contours, hierarchy = cv2.findContours(canny, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

for c in contours:
    cArea = cv2.contourArea(c)
    if cArea < 1.1*area and cArea > 0.9*area:
        break

# Find the (rotated) rectangle with the minimum area
rect = cv2.minAreaRect(c)
box = np.int0( cv2.boxPoints(rect) ) # Get vertices coordinates and transform them in integers


# Draw the box and show what we have so far
minX = min(box[:, 1])
maxX = max(box[:, 1])
minY = min(box[:, 0])
maxY = max(box[:, 0])

cropped = result[minX:maxX, minY:maxY]
croppedGray = gray[minX:maxX, minY:maxY]

cv2.drawContours(result,[box],0,(0,0,255),2)
#print(box)

cv2.imshow('Original Image', image)
cv2.imshow('Rotated Image', result)
cv2.imshow('Final Result', cropped)
cv2.imshow('Final Result (Gray)', croppedGray)
cv2.imwrite("output/cropped.png",cropped)
cv2.imwrite("output/croppedGray.png",croppedGray)
cv2.waitKey(0)

-1

Resources:
- https://docs.opencv.org/3.1.0/dd/d49/tutorial_py_contour_features.html  
- https://docs.opencv.org/4.5.2/d4/d73/tutorial_py_contours_begin.html  
- https://docs.opencv.org/3.4/dd/d49/tutorial_py_contour_features.html